# Limpieza de un CSV con data-cleaner

Se utilizan **data-cleaner** y **pandas** para codificar la limpieza de los datos de un archivo CSV. Primero se realiza una exploración de la tabla aplicando algunas reglas de limpieza y comprobando el resultado generado. Cuando este resultado es satisfactorio, se agrega la regla de limpieza a la lista codificada que luego se utilizará para generar la versión limpia del archivo.

## Inicio

In [14]:
from data_cleaner import DataCleaner

In [15]:
input_path = "audiencias-raw.csv"
output_path = "audiencias-clean.csv"

In [16]:
dc = DataCleaner(input_path)

## Exploración y descubrimiento

In [17]:
dc.df.columns

Index([u'audienciaid', u'obligadodependenciaid', u'obligadoid',
       u'obligado_cargo', u'obligado_posicion', u'obligado_apellido',
       u'obligado_nombre', u'dependenciaid', u'dependencia_descripcion',
       u'dependencia_depende', u'dependencia_posicion',
       u'dependenciadescripcion', u'rootdependenciaid',
       u'rootdependenciadescripcion', u'fechasolicitud', u'solicitanteid',
       u'solic_personajuridica', u'solic_apellido', u'solic_nombre',
       u'solic_cargo', u'tipointeresinvocadoid', u'interes_invocado',
       u'tipocaracterid', u'desc_caracter', u'representadoid',
       u'representado_cargo', u'representado_personajuridica',
       u'representado_nombre', u'representado_apellido', u'fechaaudiencia',
       u'lugar', u'objeto', u'participante', u'comentario', u'tipoestadoid',
       u'desc_estado', u'tipoestadonorealizadaid', u'desc_estado_no_realizado',
       u'sintesis', u'obligadoasesorid', u'obligadoasesorcargo',
       u'asesor_apellido', u'asesor_nombre'

## Reglas de limpieza codificadas

In [18]:
rules = [
    {
        "remover_columnas": [
            {"field": "obligadodependenciaid"},
            {"field": "obligadoid"},
            {"field": "obligado_posicion"},
            {"field": "dependenciaid"},
            {"field": "dependencia_depende"},
            {"field": "dependencia_posicion"},
            {"field": "rootdependenciaid"},
            {"field": "solicitanteid"},
            {"field": "solic_personajuridica"},
            {"field": "tipointeresinvocadoid"},
            {"field": "tipocaracterid"},
            {"field": "representadoid"},
            {"field": "tipoestadoid"},
            {"field": "tipoestadonorealizadaid"},
            {"field": "obligadoasesorid"},
            {"field": "dependenciadescripcion"},
            
        ]
    },
    {
        "nombre_propio": [
            {"field": "obligado_cargo", "keep_original": False},
            {"field": "obligado_apellido", "keep_original": False},
            {"field": "obligado_nombre", "keep_original": False},
            {"field": "dependencia_descripcion", "keep_original": False},
            {"field": "rootdependenciadescripcion", "keep_original": False},
            {"field": "solic_apellido", "keep_original": False},
            {"field": "solic_nombre", "keep_original": False},
            {"field": "solic_cargo", "keep_original": False},
            {"field": "interes_invocado", "keep_original": False},
            {"field": "desc_caracter", "keep_original": False},
            {"field": "representado_apellido", "keep_original": False},
            {"field": "desc_estado", "keep_original": False},
            {"field": "obligadoasesorcargo", "keep_original": False},
            {"field": "asesor_apellido", "keep_original": False},
            {"field": "asesor_nombre", "keep_original": False},
            {"field": "asesor_cargo", "keep_original": False},
        ]
    },
    {
        "string": [
            {"field": "obligado_cargo", "keep_original": False},
            {"field": "obligado_apellido", "keep_original": False},
            {"field": "obligado_nombre", "keep_original": False},
            {"field": "dependencia_descripcion", "keep_original": False},
            {"field": "rootdependenciadescripcion", "keep_original": False},
            {"field": "solic_apellido", "keep_original": False},
            {"field": "solic_nombre", "keep_original": False},
            {"field": "solic_cargo", "keep_original": False},
            {"field": "interes_invocado", "keep_original": False},
            {"field": "desc_caracter", "keep_original": False},
            {"field": "representado_apellido", "keep_original": False},
            {"field": "desc_estado", "keep_original": False},
            {"field": "obligadoasesorcargo", "keep_original": False},
            {"field": "asesor_apellido", "keep_original": False},
            {"field": "asesor_nombre", "keep_original": False},
            {"field": "asesor_cargo", "keep_original": False},
        ]
    },
    {
        "fecha_completa": [
            {"field": "fechaaudiencia", "time_format": "YY-MM-DD HH:mm:ss"},
        ]
    },
    {
        "fecha_simple": [
            {"field": "fechasolicitud", "time_format": "YY-MM-DD HH:mm:ss"},
        ]
    },
    {
        "renombrar_columnas": [
            {"field": "audienciaid", "new_field": "audiencia_id"},
            {"field": "rootdependenciadescripcion", "new_field": "root_dependencia_descripcion"},
            {"field": "fechasolicitud", "new_field": "fecha_solicitud"},
            {"field": "fechaaudiencia", "new_field": "fecha_audiencia"},
            {"field": "representado_personajuridica", "new_field": "representado_persona_juridica"},
            {"field": "obligadoasesorcargo", "new_field": "obligado_asesor_cargo"}
        ]
    }
]

## Limpieza

In [19]:
dc.clean(rules)

In [20]:
dc.df

,audiencia_id,obligado_cargo,obligado_apellido,obligado_nombre,dependencia_descripcion,root_dependencia_descripcion,fecha_solicitud,solic_apellido,solic_nombre,solic_cargo,...,objeto,participante,comentario,desc_estado,desc_estado_no_realizado,sintesis,obligado_asesor_cargo,asesor_apellido,asesor_nombre,asesor_cargo
0,1,Director A/C de Prevención,Macedo,Ricardo Francisco,Dirección Nacional de Prevención de las Adicci...,Secretaría de Programación Para la Prevención ...,2004-11-02,Dascola,Maria Gabriela,NaN,...,Solicitan información y apoyo institucional,No,NaN,Realizada,NaN,se derivo al representante del consejo federal...,NaN,NaN,NaN,NaN
1,2,Subsecretario de Relaciones Sanitarias E Inves...,Vizzotti,Carlos Alberto,Subsecretaría de Relaciones Sanitarias E Inves...,Ministerio de Salud y Ambiente,2004-11-03,Fernandez Madero,Matias,NaN,...,Reorganización del Instituto Panamericano de P...,tres,Posición Argentina para nuevo convenio con OPS...,Realizada,NaN,IDEM OBJETO,NaN,NaN,NaN,NaN
2,3,Secretario de Programación Para la Prevención ...,Granero,José Ramón,Sec. Programación P/ la Prevención de la Droga...,Secretaría de Programación Para la Prevención ...,2004-10-23,Guarnacci,Graciela,Presidenta (Fundacion Plural),...,Presentar la institución - Subsidio Institucional,No,No,Realizada,NaN,Presentación de la institución y sus objetivos...,NaN,NaN,NaN,NaN
3,4,Secretario de Programación Para la Prevención ...,Granero,José Ramón,Sec. Programación P/ la Prevención de la Droga...,Secretaría de Programación Para la Prevención ...,2004-10-25,Bocchio,Mirta,NaN,...,Modulo de medida de seguridad educativa y educ...,No,NaN,Realizada,NaN,Avances de los trabajos realizados por las com...,NaN,NaN,NaN,NaN
4,5,Secretario de Programación Para la Prevención ...,Granero,José Ramón,Sec. Programación P/ la Prevención de la Droga...,Secretaría de Programación Para la Prevención ...,2004-10-25,Stern,Alfredo,NaN,...,Tema adicciones,No,NaN,Realizada,NaN,Se abordaron temas relacionados a la prevenció...,NaN,NaN,NaN,NaN
5,6,Secretario de Programación Para la Prevención ...,Granero,José Ramón,Sec. Programación P/ la Prevención de la Droga...,Secretaría de Programación Para la Prevención ...,2004-10-26,Mingola,Alfredo,NaN,...,Adicciones,No,NaN,Realizada,NaN,Se abordaron temas respecto a la problematica ...,NaN,NaN,NaN,NaN
6,7,Subsecretario de Gestión de Medios,Lenzberg,Alejandro Cesar,Subsecretaría de Gestión de Medios,Jefatura de Gabinete de Ministros,2004-11-01,Gonzalez,Ruben,NaN,...,Auditoría SNMP SE,"GONZÁLEZ, Rubén",NaN,Realizada,NaN,Trabajo de auditoría,NaN,NaN,NaN,NaN
7,8,Subsecretario de Gestión de Medios,Lenzberg,Alejandro Cesar,Subsecretaría de Gestión de Medios,Jefatura de Gabinete de Ministros,2004-11-01,Ezcurra,Federico,NaN,...,Radio Rivadavia,"Ezcurra, Federico",NaN,Realizada,NaN,Presentación del medio,NaN,NaN,NaN,NaN
8,11,Secretario de Trabajo,Rial,Noemí,Secretaría de Trabajo,"Ministerio de Trabajo, Empleo y Seguridad Social",2004-11-02,Monza,Alfredo,NaN,...,RELACIONADOS AL SECTOR,ALFREDO MONZA,COMPLETAR POR ....,Realizada,NaN,COOPERACION TECNICA CON OIT,NaN,NaN,NaN,NaN
9,12,Interventor,Limeres,Manuel,"Administración Nacional de Medicamentos, Alime...",Ministerio de Salud y Ambiente,2004-10-29,Grecco,Olga,No Informa,...,presentacion directora tecnica,Dra. técnica e Interventor,NaN,Realizada,NaN,Se presentó la Sra. Directora Técnica.,NaN,NaN,NaN,NaN


In [23]:
for c in dc.df.columns:
    print(c)

audiencia_id
obligado_cargo
obligado_apellido
obligado_nombre
dependencia_descripcion
root_dependencia_descripcion
fecha_solicitud
solic_apellido
solic_nombre
solic_cargo
interes_invocado
desc_caracter
representado_cargo
representado_persona_juridica
representado_nombre
representado_apellido
fecha_audiencia
lugar
objeto
participante
comentario
desc_estado
desc_estado_no_realizado
sintesis
obligado_asesor_cargo
asesor_apellido
asesor_nombre
asesor_cargo


In [25]:
dc.df.asesor_cargo[0]

nan

In [ ]:
dc.save(output_path)